# Neuronal Networks with Spiral Data

The exercise is borrowed from https://cs231n.github.io/neural-networks-case-study/

# Import Modules

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Generate Spiral data

In [ ]:
np.random.seed(0)
N = 100 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes
X = np.zeros((N*K,D))
y = np.zeros(N*K, dtype='uint8')
for j in range(K):
    ix = range(N*j,N*(j+1))
    r = np.linspace(0.0,1,N) # radius
    t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
    X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
    y[ix] = j
fig = plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim([-1.5,1.5])
plt.ylim([-1,1])

## Activation Functions

<img src="img/activation_functions.png" title="A collection of activation functions"/>
Source: https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def dsigmoid(x, dforward):
    t = sigmoid(x)
    return np.multiply(dforward, t * (1.0 - t))

def ReLU(X, W, b):
    return np.maximum(0, np.dot(X, W) + b)
    #pass

def tanh(x):
    #pass

## Define Functions

A new activation function is used here: softmax. It is basically used to handle multiple classes. 


In [ ]:
# neural layers
def fn_layer1(X, W1, b1):
    layer1_f = np.dot(X, W1) + b1
    #hidden_layer = sigmoid(layer1_f) # Parameter to play around with activation functions
    #hidden_layer = ReLU(X, W1, b1)
    #hidden_layer = tanh(layer1_f)
    hidden_layer = layer1_f
        
    return layer1_f, hidden_layer


def fn_layer2(hidden_layer, W2, b2):
    f = np.dot(hidden_layer, W2) + b2
    
    return f


def softmax(f):
    exp_f = np.exp(f)
    probs = exp_f / np.sum(exp_f, axis=1, keepdims=True) # [N x K]
    
    return probs


def loss(X, W1, b1, W2, b2, y, reg=1e-3):
    # compute the class probabilities
    layer1_f, hidden_layer = fn_layer1(X, W1, b1)
    probs = softmax(fn_layer2(hidden_layer, W2, b2))

    # compute the loss: average cross-entropy loss and regularization
    corect_logprobs = -np.log(probs[range(num_examples),y])
    data_loss = np.sum(corect_logprobs)/num_examples
    reg_loss = 0.5*reg*np.sum(W1*W1) + 0.5*reg*np.sum(W2*W2)
    
    # compute total loss
    loss = data_loss + reg_loss
    
    return loss

## Gradient descent

<img src="img/backprop.jpg" title="Example of a neural network"/>
Source: https://www.techleer.com/articles/242-backpropagation-a-supervised-learning-neural-network-method/

In [ ]:
def gradients(X, W1, b1, W2, b2, y):
    # compute the class probabilities
    layer1_f, hidden_layer = fn_layer1(X, W1, b1) # hidden_layer is already sigmoid / ReLU / tanh etc. activated
    probs = softmax(fn_layer2(hidden_layer, W2, b2)) # get the probabilities


    # compute the gradient of function
    df = probs
    df[range(num_examples),y] -= 1
    df /= num_examples
    
    # backpropagate the gradient to the parameters
    # first backprop into parameters W2 and b2
    dW2 = np.dot(hidden_layer.T, df)
    db2 = np.sum(df, axis=0, keepdims=True)
    
    # next backprop into hidden layer
    dhidden = np.dot(df, W2.T)
    
    # backprop the sigmoid, ReLU non-linearity, tanh    # change parameteres here!
    #dhidden[hidden_layer <= 0] = 0            # derived ReLU function
    #dhidden = dsigmoid(layer1_f, dhidden)     # derived sigmoid function
    #dhidden = dtanh(layer1_f)                 # derived tanh function
    
    # finally into W1 and b1
    dW1 = np.dot(X.T, dhidden)
    db1 = np.sum(dhidden, axis=0, keepdims=True)
    
    # add regularization gradient contribution
    dW2 += reg * W2
    dW1 += reg * W1
    
    return (dW1, db1, dW2, db2)

## Initialize the parameters and perform the gradient descent

In [ ]:
# initialize parameters randomly
h = 100 # size of hidden layer; can be set individually
W1 = 0.01 * np.random.randn(D,h)
b1 = np.zeros((1,h))
W2 = 0.01 * np.random.randn(h,K)
b2 = np.zeros((1,K))


# some hyperparameters
step_size = 1e0
reg = 1e-3 # regularization strength

# gradient descent loop
num_examples = X.shape[0]
for i in range(10000):
    # Calculate average loss
    avg_loss = loss(X, W1, b1, W2, b2, y, reg=reg)
    if i % 1000 == 0:
        print ("iteration %d: loss %f" % (i, avg_loss))

    # Compute the gradients
    dW1, db1, dW2, db2 = gradients(X, W1, b1, W2, b2, y)

    # perform parameter updates
    W1 += -step_size * dW1
    b1 += -step_size * db1
    W2 += -step_size * dW2
    b2 += -step_size * db2

## Test the accuracy of the training model

In [ ]:
# evaluate training set accuracy
layer1_f, hidden_layer = fn_layer1(X, W1, b1)
scores = fn_layer2(hidden_layer, W2, b2)

predicted_class = np.argmax(scores, axis=1)
print ('training accuracy: %.2f' % (np.mean(predicted_class == y)))

## Visualization

In [ ]:
# if this one doesn't work, try the second!

colors = ['r','y','b']
nx,ny = 500,500
xx,yy = np.meshgrid(np.linspace(-1.5, 1.5, nx), np.linspace(-1, 1, ny))
x_grid = np.hstack((xx.flatten().reshape(nx*ny,1), yy.flatten().reshape(nx*ny,1)))

layer1_f, hidden_layer = fn_layer1(x_grid, W1, b1)
layer2_f = fn_layer2(hidden_layer, W2, b2)

y_pred = np.argmax(layer2_f, axis=1)

plt.scatter(xx, yy, c=y_pred, s=40, marker='s', edgecolor='none', cmap=plt.cm.Spectral)
plt.scatter(X[:,0], X[:, 1], c=[colors[l] for l in y], s=40)
plt.axis('equal')
x1,x2,y1,y2 = plt.axis()
plt.xlim([-1.5,1.5])
plt.ylim([-1,1])
plt.show()

In [ ]:
# plot the resulting classifier
h = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = np.dot(np.maximum(0, np.dot(np.c_[xx.ravel(), yy.ravel()], W1) + b1), W2) + b2
Z = np.argmax(Z, axis=1)
Z = Z.reshape(xx.shape)
fig = plt.figure()
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())